# Redes neuronales hibridas  para clasificación multiple


In [2]:
# Install the relevant packages.
#%pip install --upgrade pip
%pip install torch torchvision torchaudio
#%pip install cudaq -> ya viene instalado en el braket de AWS

  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB ?  0:02:45eta 0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 23.2 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 217.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 76.8 MB/s  0:00:016m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 20.7 MB/s  0:00:16m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 62.3 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 84.8 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 47.3 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 44.1 MB/s  0:00:06m0:

In [3]:
# Check installed clasico
import sys
import numpy as np
import matplotlib
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

import torch, torchvision, torchaudio
print("torch:", torch.__version__)
print("vision:", torchvision.__version__)
print("audio:", torchaudio.__version__)

Python version: 3.10.19 | packaged by conda-forge | (main, Oct 22 2025, 22:29:10) [GCC 14.3.0]
NumPy version: 1.26.4
Matplotlib version: 3.10.7
torch: 2.9.1+cu128
vision: 0.24.1+cu128
audio: 2.9.1+cu128


In [4]:

import cudaq
print(f"CUDAQ version: {cudaq.__version__}")
print(f"Running on target: {cudaq.get_target().name}")

CUDAQ version: CUDA-Q Version 0.12.0 (https://github.com/NVIDIA/cuda-quantum 6adf92bcda4df7465e4fe82f1c8f782ae69d8bd2)
Running on target: qpp-cpu


In [ ]:
import sys
import cudaq

print(f"Running on target {cudaq.get_target().name}")
qubit_count = int(sys.argv[1]) if 1 < len(sys.argv) else 2


@cudaq.kernel
def kernel():
    qubits = cudaq.qvector(qubit_count)
    h(qubits[0])
    for i in range(1, qubit_count):
        x.ctrl(qubits[0], qubits[i])
    mz(qubits)


result = cudaq.sample(kernel)
print(result)  # Example: { 11:500 00:500 }

In [ ]:
# importar las librerías necesarias
import cudaq
from cudaq import spin

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Para asegurar que los resultados sean reproducibles
torch.manual_seed(42)

cudaq.set_random_seed(44)

In [ ]:
# Configurar el dispositivo
# Set CUDAQ and PyTorch to run on either CPU or GPU.

device = torch.device('cpu')
cudaq.set_target("qpp-cpu")

#cudaq.set_target("nvidia")
#device = torch.device("cuda:0")

## Descripción del Conjunto de Datos MNIST

- *¿Qué es?:* MNIST (Modified National Institute of Standards and Technology database) es una gran base de datos de dígitos escritos a mano, del 0 al 9.
- *Contenido:* Contiene 70,000 imágenes en escala de grises.
- *Conjunto de entrenamiento:* 60,000 imágenes.
- *Conjunto de prueba:* 10,000 imágenes.
- *Formato de imagen:* Cada imagen tiene un tamaño de 28x28 píxeles.
- *Uso común:* Es considerado el "Hola, Mundo" de la visión por computadora y el aprendizaje profundo. Se utiliza para entrenar y probar algoritmos de clasificación de imágenes.


Paso 2: Cargar, Transformar y Previsualizar los Datos
torchvision nos facilita la descarga y preparación de datasets.

Transformaciones: Convertimos las imágenes a tensores de PyTorch y las normalizamos. La normalización (ajustar los valores de los píxeles para que tengan una media de 0.5 y una desviación estándar de 0.5) ayuda a que el modelo entrene más rápido y de forma más estable.
Descarga: Descargamos los conjuntos de entrenamiento y prueba. FashionMNIST ya viene separado en train y test.

In [ ]:
# 1. Definir las transformaciones para las imágenes
# - transforms.ToTensor() convierte la imagen (PIL) a un Tensor de PyTorch.
# - transforms.Normalize() ajusta los valores del tensor para que tengan una media y desviación estándar específicas.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # Media y Desviación Estándar para un solo canal (escala de grises)
])

# 2. Descargar los datasets de entrenamiento y prueba
train_full_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform
)

print(f"Tamaño total del dataset de entrenamiento: {len(train_full_dataset)}")
print(f"Tamaño del dataset de prueba: {len(test_dataset)}")


In [ ]:
# --- MODIFICACIÓN: Usar solo 1000 muestras para una prueba rápida ---
from torch.utils.data import Subset

# 1. Creamos un subconjunto con las primeras 1000 imágenes del dataset de entrenamiento completo
n_samples_prueba = 1000
train_subset_for_testing = Subset(train_full_dataset, range(n_samples_prueba))

# 2. Ahora, basamos la división 80/20 en este subconjunto de 1000 muestras
train_size = int(0.8 * len(train_subset_for_testing)) # 80% de 1000 = 800
val_size = len(train_subset_for_testing) - train_size # 20% de 1000 = 200

# Dividimos el dataset de entrenamiento
train_dataset, val_dataset = random_split(train_subset_for_testing, [train_size, val_size])

print(f"Tamaño del subconjunto de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del subconjunto de validación: {len(val_dataset)}")

# Crear los DataLoaders
# Los DataLoaders nos permiten iterar sobre los datos en lotes (batches)
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
# Clases de MNIST (dígitos del 0 al 9)
classes = tuple(str(i) for i in range(10))

# Función para mostrar imágenes (sin cambios)
def imshow(img):
    img = img / 2 + 0.5  # Des-normalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Obtener un lote de imágenes de entrenamiento
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Mostrar las primeras 8 imágenes y sus etiquetas
print("--- Previsualización de Datos (MNIST) ---")
imshow(torchvision.utils.make_grid(images[:8]))
print('Etiquetas: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Bloque convolucional para extracción de características
        self.conv_stack = nn.Sequential(
            # Entrada: 1x28x28 -> Salida: 32x26x26
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3),
            nn.ReLU(),
            # Entrada: 32x26x26 -> Salida: 32x13x13
            nn.MaxPool2d(kernel_size=2),
            # Entrada: 32x13x13 -> Salida: 64x11x11
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            # Entrada: 64x11x11 -> Salida: 64x5x5
            nn.MaxPool2d(kernel_size=2)
        )
        # Aplanador y clasificador
        self.flatten = nn.Flatten()
        self.fc_stack = nn.Sequential(
            # Previene el sobreajuste
            nn.Dropout(0.5),
            # Mapea las características a las 10 clases de salida
            # La entrada es 64*5*5 = 1600
            nn.Linear(64 * 5 * 5, 10)
        )

    def forward(self, x):
        # Pasa la imagen por las capas convolucionales
        x = self.conv_stack(x)
        # Aplana el resultado para la capa densa
        x = self.flatten(x)
        # Pasa por el clasificador final
        logits = self.fc_stack(x)
        return logits

# Instanciamos el modelo y lo movemos al dispositivo (GPU/CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNN().to(device)
print("\n--- Arquitectura del Modelo ---")
print(model)



--- Arquitectura del Modelo ---
CNN(
  (conv_stack): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc_stack): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=1600, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Pérdida (loss): {loss:>7f}  [{current:>5d}/{size:>5d}]")

def validation_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"Resultado: \n Precisión (Accuracy): {(100*correct):>0.1f}%, Pérdida promedio: {val_loss:>8f} \n")

# Bucle principal de entrenamiento
epochs = 10
print("\n--- Iniciando Entrenamiento con MNIST ---")
for t in range(epochs):
    print(f"Época {t+1}\n-------------------------------")
    print("Entrenando...")
    train_loop(train_loader, model, loss_fn, optimizer)
    print("Validando...")
    validation_loop(val_loader, model, loss_fn)
print("¡Entrenamiento finalizado!")


In [ ]:
print("\n--- Evaluando con el conjunto de Prueba (Test) ---")
validation_loop(test_loader, model, loss_fn)
